EL dataset puede descargarse desde acá https://www.kaggle.com/datasets/imsparsh/flowers-dataset?resource=download

In [1]:
import gdown

# ID del archivo de Google Drive (extraído de la URL)
file_id = "15dU4qL6J2CS9iAp36JxBDZOAI7pH6y0m"
# URL de descarga directa
url = f"https://drive.google.com/uc?id={file_id}"

# Nombre del archivo a guardar
filename = "archive.zip"

# Descargar el archivo
gdown.download(url, filename, quiet=False)

print(f"Archivo descargado: {filename}")


Downloading...
From (original): https://drive.google.com/uc?id=15dU4qL6J2CS9iAp36JxBDZOAI7pH6y0m
From (redirected): https://drive.google.com/uc?id=15dU4qL6J2CS9iAp36JxBDZOAI7pH6y0m&confirm=t&uuid=2ea782e3-237a-4ee3-98cf-13c842854131
To: /content/archive.zip
100%|██████████| 215M/215M [00:13<00:00, 16.2MB/s]

Archivo descargado: archive.zip


In [2]:
import zipfile
import os

# Nombre del archivo descargado
filename = "archive.zip"

# Ruta donde se descomprimirá el contenido
extract_path = "./flowers_dataset"

# Crear el directorio si no existe
os.makedirs(extract_path, exist_ok=True)

# Descomprimir el archivo
with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Archivo descomprimido en: {extract_path}")


Archivo descomprimido en: ./flowers_dataset


In [3]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import DataLoader
import os
import time

# Configuración de parámetros
batch_size = 32
learning_rate = 0.001
num_epochs = 10

# Ruta del dataset
data_dir = './flowers_dataset'

# Transformaciones para el dataset de entrenamiento
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Cargar dataset de entrenamiento
train_dataset = datasets.ImageFolder(root=os.path.join(data_dir, 'train'), transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

# Definir el modelo ResNet-18 sin pesos preentrenados
model = models.resnet18(weights=None)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, len(train_dataset.classes))  # Ajustar la capa final

# Mover el modelo a GPU si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Definir la función de pérdida y el optimizador
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

# Entrenamiento del modelo
start_time = time.time()  # Tiempo de inicio
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    epoch_start_time = time.time()  # Tiempo de inicio de la época

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Update running loss
        running_loss += loss.item() * inputs.size(0)

        # Calculate accuracy
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_samples += labels.size(0)

    epoch_loss = running_loss / len(train_dataset)
    epoch_accuracy = correct_predictions / total_samples

    epoch_end_time = time.time()  # Tiempo de fin de la época
    epoch_duration = epoch_end_time - epoch_start_time
    avg_epoch_duration = (epoch_end_time - start_time) / (epoch + 1)
    remaining_time = avg_epoch_duration * (num_epochs - (epoch + 1))

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}')
    print(f'Epoch Time: {epoch_duration:.2f}s, Estimated Time Remaining: {remaining_time:.2f}s')

print('Entrenamiento completado.')

# Guardar el modelo entrenado
torch.save(model.state_dict(), 'resnet18_flower_model.pth')


Epoch 1/10, Loss: 1.3321, Accuracy: 0.4421
Epoch Time: 5.02s, Estimated Time Remaining: 45.22s
Epoch 2/10, Loss: 1.0473, Accuracy: 0.5863
Epoch Time: 3.31s, Estimated Time Remaining: 33.34s
Epoch 3/10, Loss: 0.9385, Accuracy: 0.6351
Epoch Time: 3.27s, Estimated Time Remaining: 27.07s
Epoch 4/10, Loss: 0.8699, Accuracy: 0.6650
Epoch Time: 3.70s, Estimated Time Remaining: 22.96s
Epoch 5/10, Loss: 0.8025, Accuracy: 0.6908
Epoch Time: 3.50s, Estimated Time Remaining: 18.81s
Epoch 6/10, Loss: 0.7590, Accuracy: 0.6992
Epoch Time: 3.40s, Estimated Time Remaining: 14.81s
Epoch 7/10, Loss: 0.7001, Accuracy: 0.7353
Epoch Time: 3.42s, Estimated Time Remaining: 10.98s
Epoch 8/10, Loss: 0.6829, Accuracy: 0.7393
Epoch Time: 3.44s, Estimated Time Remaining: 7.27s
Epoch 9/10, Loss: 0.6374, Accuracy: 0.7600
Epoch Time: 3.37s, Estimated Time Remaining: 3.60s
Epoch 10/10, Loss: 0.5714, Accuracy: 0.7848
Epoch Time: 3.32s, Estimated Time Remaining: 0.00s
Entrenamiento completado.
